In [1]:
import os
import cv2
import math
import json
import shutil
train_dir='train/'
labels_dir='train_labels/'
val_dir='val/'
val_labels='val_labels/'

In [2]:
super_category = 'gta_car'
class_names =  ['car']
dataset_info = {'info': {'description': 'gta_traindataset', 'url': 'https://drive.google.com/file/d/1L4XU3rCCEwQqwIk1V419bxxZuOtsOnfQ/view', 'version': '0.0',
                'year': 2022, 'contributor': 'nctu'}, 
                'images': [], 'annotations': [], 
                'categories': [{'supercategory': super_category, 'id': i+1, 'name': cls} for i, cls in enumerate(class_names)]}
train_dataset = {}
train_dataset.update(dataset_info)
val_dataset_info = {'info': {'description': 'gta_valdataset', 'url': 'https://drive.google.com/file/d/1L4XU3rCCEwQqwIk1V419bxxZuOtsOnfQ/view', 'version': '0.0',
                'year': 2022, 'contributor': 'nctu'}, 
                'images': [], 'annotations': [], 
                'categories': [{'supercategory': super_category, 'id': i+1, 'name': cls} for i, cls in enumerate(class_names)]}
val_dataset={}
val_dataset.update(val_dataset_info)

In [3]:
def yolo2coco(box_info, img_size):
    x_min = (box_info[0] - box_info[2]/2) * img_size[1]
    x_max = (box_info[0] + box_info[2]/2) * img_size[1]
    y_min = (box_info[1] - box_info[3]/2) * img_size[0]
    y_max = (box_info[1] + box_info[3]/2) * img_size[0]
    
    return x_min, x_max, y_min, y_max
ann_id=1
images_dir = os.path.join(train_dir )

width=1920
height=1080
for idx, image_file in enumerate(os.listdir(images_dir)):
    #height, width, _ = cv2.imread(os.path.join(images_dir, image_file)).shape
    train_dataset['images'].append({'file_name': image_file, 'width': 1920, 'height': 1080, 'id': idx+1})
    image_txt = image_file.replace('.jpg','.txt').replace('.png','.txt')
    with open(os.path.join(labels_dir, image_txt), 'r') as f:
        label_info = f.readlines()
        for i_label in label_info:
            label = i_label.split()
            class_id = int(label[0])+1
            x = float(label[1])
            y = float(label[2])
            w = float(label[3])
            h = float(label[4])
            box_info = [x, y, w, h]
            img_size = [height, width]
            x_min, x_max, y_min, y_max = yolo2coco(box_info, img_size)
            box_w = max(0, x_max-x_min)
            box_h = max(0, y_max-y_min)
            train_dataset['annotations'].append({'segmentation': [], 'area': box_w * box_h, 'iscrowd': 0, 'image_id': idx+1, 
                                           'bbox': [x_min, y_min, box_w, box_h], 'category_id': class_id, 'id': ann_id
                                          })
            ann_id += 1



    # if (idx%10==0):
    #     print(idx)


In [4]:
val_annid=1
val_image_dir=os.path.join(val_dir)
for idx, image_file in enumerate(os.listdir(val_image_dir)):
    val_dataset['images'].append({'file_name': image_file, 'width': 1920, 'height': 1080, 'id': idx+1})
    image_txt = image_file.replace('.jpg','.txt').replace('.png','.txt')
    with open(os.path.join(val_labels, image_txt), 'r') as f:
        label_info = f.readlines()
        for i_label in label_info:
            label = i_label.split()
            class_id = int(label[0])+1
            x = float(label[1])
            y = float(label[2])
            w = float(label[3])
            h = float(label[4])
            box_info = [x, y, w, h]
            img_size = [height, width]
            x_min, x_max, y_min, y_max = yolo2coco(box_info, img_size)
            box_w = max(0, x_max-x_min)
            box_h = max(0, y_max-y_min)
            val_dataset['annotations'].append({'segmentation': [], 'area': box_w * box_h, 'iscrowd': 0, 'image_id': idx+1, 
                                           'bbox': [x_min, y_min, box_w, box_h], 'category_id': class_id, 'id': val_annid
                                          })
            val_annid += 1
        
    

In [5]:
#print(((train_dataset['annotations'][1])))
#print((val_dataset['annotations']))
rootdir=''
annotation_dir = os.path.join(rootdir, 'annotations')
if not os.path.exists(annotation_dir):
    os.makedirs(annotation_dir,exist_ok=True)
with open(os.path.join(annotation_dir, 'train.json'), 'w') as json_file:
  json.dump(train_dataset, json_file)

with open(os.path.join(annotation_dir, 'val.json'), 'w') as json_file:
  json.dump(val_dataset, json_file)